In [1]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# Creating a Vertex Pipeline to extract training data

This notebook (the second in a five-part series) creates a Vertex AI pipeline that scrapes images from an online source (e.g. Reddit) and stores the image metadata in Firestore. Here, you will build a pipeline that 

This notebook covers the following steps:

1. Creating a pipeline component to collect images from Reddit
1. Creating a pipeline component to store images in Cloud Storage
1. Creating a pipeline component to store metadata about the images in Firestore.

### Set IAM permissions

When you run a notebook on Vertex Workbench, the notebook runs in a Compute Engine context that has its own service account. You will need to give your service account IAM permissions to access Secret Manager before you can use it (in a pipeline).



### Enable the Cloud resources

For this notebook, you must have a Google Cloud project with the following resources:

+ A Cloud Storage bucket
+ The following APIs enabled:
  - Cloud Firestore
  - Vertex AI
  - Storage
  - Secret Manager
  
If you completed the [first](1_firestore.ipynb) notebook in this series, you should have these APIs already enabled.

In [17]:
# Get your GCP project id from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID=shell_output[0]
print("Project ID: ", PROJECT_ID)

Project ID:  fantasymaps-334622


In [120]:
BUCKET = "fantasy-maps" # Google Cloud Storage bucket
COLLECTION_NAME = "FantasyMapsTest" # Firestore collection name
LOCATION = "us-central1"
GCS_PREFIX = "ScrapedData"
SUBREDDIT_NAME = "battlemaps"
LIMIT=300
MODEL_ID = "4304645197347684352"

### Install the required Python libraries

In [191]:
! rm -rfd requirements.txt

In [192]:
%%writefile requirements.txt
google-cloud-secret-manager
google-cloud-aiplatform
google-cloud-pipeline-components>=1.0.30
kfp
praw
pandas
spacy
pillow

Writing requirements.txt


In [193]:
! pip install -r requirements.txt

     |████████████████████████████████| 817 kB 4.8 MB/s            
     |████████████████████████████████| 2.3 MB 67.6 MB/s            
     |████████████████████████████████| 409 kB 85.8 MB/s            
     |████████████████████████████████| 47 kB 5.8 MB/s             
     |████████████████████████████████| 235 kB 64.1 MB/s            
     |████████████████████████████████| 120 kB 81.4 MB/s            
     |████████████████████████████████| 1.0 MB 42.5 MB/s            
     |████████████████████████████████| 115 kB 47.2 MB/s            
     |████████████████████████████████| 217 kB 79.9 MB/s            
     |████████████████████████████████| 62 kB 510 kB/s             
     |████████████████████████████████| 233 kB 59.5 MB/s            
     |████████████████████████████████| 4.8 MB 51.9 MB/s            
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
  Attempting

In [22]:
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 5.1 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Create a custom Reddit pipelines component

The pipeline and all it components need to be compiled into a runnable format. We use the Kubeflow Pipelines (`kfp`) SDK to create this uploadable pipelines file.

In [23]:
from typing import NamedTuple

import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient

from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

Now we can define the pipeline. For this component, we are going to store the `pandas.DataFrame` that we compose from the Redit posts as a CSV file on Cloud Storage. We'll pass the URI of this Storage file onto the next piece of the pipeline.

In [94]:
@component(packages_to_install=["praw",
                                "google-cloud-secret-manager",
                                "google-cloud-storage",
                                "numpy",
                                "pandas",
                                "spacy"])
def reddit(
    secret_name: str,
    subreddit_name: str,
    gcs_bucket_name: str,
    gcs_prefix_name: str,
    project_id: str,
    limit: int,
) -> str:
    from datetime import datetime
    import numpy as np
    import pandas as pd
    import praw
    import re
    
    from google.cloud import storage

    def get_reddit_credentials(project_id):
        """Gets the Reddit API key out of Secrets Manager
    
        Arguments:
            project_id (str): the current project ID

        Returns:
            JSON object (dict)
        """
        from google.cloud import secretmanager
        import json

        client = secretmanager.SecretManagerServiceClient()

        secret_resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/1"
        response = client.access_secret_version(request={"name": secret_resource_name})
        payload = response.payload.data.decode("UTF-8")

        return json.loads(payload)
    
    def get_reddit_posts(reddit_credentials, subreddit_name, limit):
        """Gets posts from a subreddit.

        Arguments:
            reddit_credentials (dict): a dictionary with client_id, secret, and user_agent
            subreddit_name (str): the name of the subreddit to scrape posts from
            limit (int): the maximum number of posts to grab

        Returns:
            List of Reddit API objects
        """
        import praw

        reddit = praw.Reddit(client_id=reddit_credentials["client_id"], 
                     client_secret=reddit_credentials["secret"],
                     user_agent=reddit_credentials["user_agent"])

        return reddit.subreddit(subreddit_name).hot(limit=limit)

    def convert_posts_to_dataframe(posts, columns):
        """Converts a sequence of Reddit API post objects into a pandas.DataFrame.
        
        Arguments:
            posts (list(praw.Post)): the posts from Reddit
            columns (list(str)): the column headings for the Dataframe
        
        Returns:
            A pandas.Dataframe
        """
        import numpy as np
        import pandas as pd

        filtered_posts = [[s.title, s.selftext, s.id, s.url] for s in posts]
        filtered_posts = np.array(filtered_posts)
        reddit_posts_df = pd.DataFrame(filtered_posts,
                                   columns=columns)

        return reddit_posts_df
    
    COLUMNS = ['Title', 'Post', 'ID', 'URL']
    
    # Get the data from Reddit
    credentials = get_reddit_credentials(project_id=project_id)
    posts = get_reddit_posts(reddit_credentials=credentials, subreddit_name=subreddit_name,
                             limit=limit)
    
    reddit_posts_df = convert_posts_to_dataframe(posts=posts, columns=COLUMNS)
    
    # Remove all of the posts that don't meet our criteria
    import re
    jpg_df = reddit_posts_df[(reddit_posts_df["URL"].str.contains("jpg")) &
                             (reddit_posts_df["Title"].str.contains(pat = "\d+x\d"))]
    
    # Save the dataframe as CSV in Storage
    csv_str = jpg_df.to_csv()
    
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(gcs_bucket_name)
    
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    
    csv_file_uri = f"{gcs_prefix_name}/_reddit-scraped-{subreddit_name}-{timestamp}.csv"
    
    file_blob = bucket.blob(csv_file_uri)
    file_blob.upload_from_string(csv_str)
    
    return csv_file_uri
    

## Create the Cloud Storage component

In this next component, we need to store any unique map images that we have picked up from the scraping. However, we need to validate that these images are useful training data before we process them and store their metadata in Firestore.

To validate the images, we will do the following:

1. Ensure that we don't already have the image in Firestore
2. Use a pre-trained, earlier version of our model to infer the existence of gridlines on the image

We'll do the first step in validation in the `storage` component. The second step (using an existing model to validate the usefulness of the images) will require using batch predictions on Vertex AI; we'll create another component to handle that part.

In [138]:
from typing import NamedTuple

@component(packages_to_install=["spacy",
                                "google-cloud-firestore",
                                "google-cloud-storage",
                                "pandas",
                                "jsonlines"])
def storage(
    project_id: str,
    location_name: str,
    gcs_bucket_name: str,
    gcs_prefix_name: str,
    collection_name: str,
    csv_input_file: str,
) -> NamedTuple(
    "outputs",
    [
        ("batch_predict_file_uri", str),
        ("posts_csv_file", str),
    ]
):
    
    from google.cloud import firestore
    from google.cloud import storage
    
    import base64
    from datetime import datetime
    from io import BytesIO
    import jsonlines
    import pandas as pd
    
    import spacy
    spacy.cli.download("en_core_web_sm")
    spacy.prefer_gpu()
    nlp = spacy.load("en_core_web_sm")
    
    def make_nice_filename(name):
        """Converts Reddit post title into a meaningful(ish) filename.

        Arguments:
            name (str): title of the post

        Returns:
            String. Format is `<adj.>-<nouns>.<cols>x<rows>.jpg`
        """
        import re

        dims = re.findall("\d+x\d+", name)
        if len(dims) is 0:
            return ""

        dims = dims[0].split("x")
        if len(dims) is not 2:
            return ""

        tokens = get_tokens(name)
        new_name = name.lower()[:30]

        if len(tokens) > 0:
            tokens = tokens[:6] # Arbitrarily keep new names to six words or less
            new_name = "_".join(tokens)

        return f"{new_name}.{dims[0]}x{dims[1]}.jpg"

    def get_tokens(title):
        """Analyzes a post for nouns, proper nouns, and adjectives.

        Arguments:
            title (str): title of the post

        Returns:
            List of string. Words to use in a filename.    
        """
        import spacy

        POS = ["PROPN", "NOUN", "ADJ"]
        words = []

        tokens = nlp(title)
        for t in tokens:
            pos = t.pos_

            if pos in POS:
                words.append(t.text.lower())

        return words
    def convert_image_to_hash(content):
        """Convert image data to hash value (str).

        Arguments:
            content (byte array): the image

        Return:
            The image hash value as a string.
        """
        import hashlib

        sha1 = hashlib.sha1()
        jpg_hash = sha1.update(content)
        jpg_hash = sha1.hexdigest()

        return jpg_hash
    
    def download_image(url):
        """Download an image from the internet to local file system.

        Arguments:
            url (str): the image to download

        Returns:
            Bool. Indicates whether downloading the image was successful.
        """
        import requests

        r = requests.get(url, stream=True)
        if r.status_code == 200:
            r.raw.decode_content = True
            
            hsh = convert_image_to_hash(r.content)
            return (r.content, hsh)
    
    # Begin pipeline
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(gcs_bucket_name)

    firestore_client = firestore.Client(project=project_id)
    collection_ref = firestore_client.collection(collection_name)

    blob = bucket.blob(csv_input_file)
    csv_bytes = blob.download_as_string()
    csv_buffer = BytesIO(csv_bytes)

    jpg_df = pd.read_csv(csv_buffer)
    batch_prediction_inputs = []
    
    for i, row, in jpg_df.iterrows():
        url = row["URL"]
        title = row["Title"]
        
        content, hsh = download_image(url)

        # Check whether we already have this image
        doc_ref = collection_ref.document(hsh)
        doc_ref = doc_ref.get()
        if not doc_ref.exists:
            continue
        
        file_name = make_nice_filename(title)
                
        img_gcs_uri = f"gs://{gcs_bucket_name}/{gcs_prefix_name}/{file_name}"
        blob_name = f"{gcs_prefix_name}/{file_name}"

        file_blob = bucket.blob(blob_name)
        image_buffer = BytesIO(content)

        # Get image grid metadata
        file_blob.upload_from_file(image_buffer)
        jpg_df.at[i, "gcsURI"] = img_gcs_uri
        jpg_df.at[i, "UID"] = hsh
        
        batch_prediction_inputs.append({
            "content": img_gcs_uri,
            "mimeType": "image/jpeg",
        })

    # Save the dataframe as CSV in Storage (again)
    csv_str = jpg_df.to_csv()
    file_blob = bucket.blob(csv_input_file)
    file_blob.upload_from_string(csv_str)
    
    # Create batch prediction input file
    bpi = BytesIO()
    writer = jsonlines.Writer(bpi)
    writer.write_all(batch_prediction_inputs)
    writer.close()
    
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    bpi_gcs_path = f"{gcs_prefix_name}/_batch_prediction_input_{timestamp}.jsonl"
    batch_prediction_input_file = f"gs://{gcs_bucket_name}/{bpi_gcs_path}"
    
    bpi_str = str(bpi.getvalue(), encoding="UTF8")
    bpi_blob = bucket.blob(bpi_gcs_path)
    bpi_blob.upload_from_string(bpi_str)
    
    return (batch_prediction_input_file, csv_input_file)

## Create the Firestore component

In [125]:
from typing import NamedTuple

@component(packages_to_install=["Pillow",
                                "google-cloud-firestore",
                                "google-cloud-storage",
                                "numpy",
                                "pandas"])
def firestore(
    subreddit_name: str,
    collection_name: str,
    gcs_bucket_name: str,
    gcs_prefix_name: str,
    csv_input_file: str,
    project_id: str,
) -> NamedTuple(
    "Outputs",
    [
        ("batch_predict_file_uri", str),
        ("bp_inputs_count", int),
    ]
):
    
    from datetime import datetime
    import hashlib
    from io import BytesIO
    import json
    import pandas as pd
    from PIL import Image
    import re
    import requests
    import shutil

    from google.cloud import firestore
    from google.cloud import storage

    def make_nice_filename(name, *, rows=None, cols=None):
        regex = "[\s|\(|\"|\)]"
        new_name = re.sub(regex, "_", name)
        new_name = new_name.lower()[:30]
        new_name = new_name.replace("__", "_")
        
        if rows is not None and cols is not None:
            new_name += f".{cols}x{rows}"
        return f"{new_name}.jpg"


    def create_vtt_json(content, title):
        img = Image.open(BytesIO(content))
        w, h = img.size

        dims = re.findall("\d+x\d+", title)
        if len(dims) is 0:
            return None

        dims = dims[0].split("x")

        if len(dims) is not 2:
            return None

        cols = int(dims[0])
        rows = int(dims[1])

        cell_w = w / rows
        cell_h = h / cols
        if cell_w != cell_h:
            return None

        return {
            "cols": cols,
            "rows": rows,
            "imageWidth": w,
            "imageHeight": h,
            "cellOffsetX": 0,
            'cellOffsetY': 0, 
            'cellWidth': cell_w, 
            'cellHeight': cell_h, 
        }

    def compute_bboxes(vtt_data):
        bboxes = []

        cols = vtt_data["cols"]
        rows = vtt_data["rows"]

        for x in range(1, cols):
            for y in range(1, rows):
               x_min_tmp = vtt_data["cellOffsetX"] + (vtt_data["cellWidth"] * x) - 2
               x_max_tmp = x_min_tmp + vtt_data["cellWidth"] + 4
               y_min_tmp = vtt_data["cellOffsetY"] + (vtt_data["cellHeight"] * y) - 2
               y_max_tmp = y_min_tmp + vtt_data["cellHeight"] + 4

               x_min_train = x_min_tmp / vtt_data["imageWidth"]
               x_max_train = x_max_tmp / vtt_data["imageWidth"]
               y_min_train = y_min_tmp / vtt_data["imageHeight"]
               y_max_train = y_max_tmp / vtt_data["imageHeight"]

               bboxes.append({
                   "xMin": x_min_train,
                   "yMin": y_min_train,
                   "xMax": x_max_train,
                   "yMax": y_max_train,
                   "displayName": "cell"
               })

        return bboxes

    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(gcs_bucket_name)

    firestore_client = firestore.Client(project=project_id)
    collection_ref = firestore_client.collection(collection_name)

    blob = bucket.blob(csv_input_file)
    csv_bytes = blob.download_as_string()
    csv_buffer = BytesIO(csv_bytes)

    jpg_df = pd.read_csv(csv_buffer)

    hashes = [None] * len(jpg_df.index)
    jpg_df.insert(1, "HashId", hashes, True)
    jpg_df.insert(6, "GcsURI", hashes, True)

    # Concatenate string of batch prediction inputs
    bp_inputs = ""
    bp_inputs_count = 0
    
    # Iterate over JPG URIs, download them in batches, convert to sha values
    for i, r in jpg_df.iterrows():
        jpg_url = r["URL"]
        title = r["Title"]

        req = requests.get(jpg_url, stream=True)
        if req.status_code == 200:
            req.raw.decode_content = True
            sha1 = hashlib.sha1()
            jpg_hash = sha1.update(req.content)
            jpg_hash = sha1.hexdigest()

            jpg_df["HashId"][i] = jpg_hash
            #print(f"Index {i}, hash {jpg_hash}")
            hashes.append(jpg_hash)

            # Try to fetch each document from Firestore. If it does not exist,
            # overwrite and download the image.
            doc_ref = collection_ref.document(jpg_hash)
            doc = doc_ref.get()
            if not doc.exists:

                img_data = create_vtt_json(req.content, title)
                
                if img_data is not None:
                    file_name = make_nice_filename(title,
                                                   rows=img_data["rows"],
                                                   cols=img_data["cols"])
                else:
                    file_name = make_nice_filename(title)
                
                img_gcs_uri = f"gs://{gcs_bucket_name}/{gcs_prefix_name}/{file_name}"
                blob_name = f"{gcs_prefix_name}/{file_name}"

                file_blob = bucket.blob(blob_name)
                image_buffer = BytesIO(req.content)

                # Get image grid metadata
                #img_data = create_vtt_json(req.content, title)
                print(img_data)

                file_blob.upload_from_file(BytesIO(req.content))

                data = {
                    u"filename": file_name,
                    u"gcsURI": img_gcs_uri,
                    u"source": gcs_prefix_name,
                    u"userId": "None",
                    u"sourceUrl": jpg_url,
                }

                if img_data is not None:
                    bboxes = compute_bboxes(img_data)
                    data["vtt"] = img_data
                    data["computedBBoxes"] = bboxes

                    doc_ref.set(data)
                    print(f"Set data: {data}")
                    bp_inputs += json.dumps({ "content": img_gcs_uri, "mimeType": "image/jpeg"})
                    bp_inputs += "\n"
                    bp_inputs_count = bp_inputs_count + 1

    # No fresh JPGs in this scraping; return empty string
    if bp_inputs is "":
        print("no inputs")
        return ("", 0)

    print(f"First ten: {jpg_df.head(10)}")

    # Save the batch_predict file
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S") 
    batch_predict_file_uri = f"gs://{gcs_bucket_name}/{gcs_prefix_name}/bp_input_{timestamp}.jsonl"

    bp_blob_name = f"{gcs_prefix_name}/bp_input_{timestamp}.jsonl"
    bp_blob = bucket.blob(bp_blob_name)

    bp_blob.upload_from_string(bp_inputs)

    return (batch_predict_file_uri, bp_inputs_count)  

## Build a simple pipeline

In [198]:
@dsl.pipeline(
    name="reddit-scraper-pipeline",
    description="Gets data from a subreddit",
    pipeline_root=f"gs://{BUCKET}/pipeline_root",
)
def reddit_pipeline(
    collection_name: str = COLLECTION_NAME,
    secret_name: str = "reddit-api-key",
    subreddit_name: str = SUBREDDIT_NAME,
    gcs_bucket: str = BUCKET,
    gcs_prefix: str = GCS_PREFIX,
    project_id: str = PROJECT_ID,
    location: str = LOCATION,
    limit: int = LIMIT,
    model_id: str = MODEL_ID,
):
    
    # Get the images from Reddit
    reddit_op = reddit(
        secret_name=secret_name,
        subreddit_name=subreddit_name,
        gcs_bucket_name=gcs_bucket,
        gcs_prefix_name=gcs_prefix,
        project_id=project_id,
        limit=limit,
    )
    
    reddit_csv_file = reddit_op.output
    
    # Store the new images on Cloud Storage
    storage_op = storage(
        project_id=project_id,
        location_name=location,
        gcs_bucket_name=gcs_bucket,
        gcs_prefix_name=gcs_prefix,
        collection_name=collection_name,
        csv_input_file=reddit_csv_file,
    )
    

In [199]:
compiler.Compiler().compile(
    pipeline_func=reddit_pipeline, package_path="artifacts/reddit_scraper_pipeline_job.json"
)

In [200]:
api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=LOCATION,
)

When we run the pipeline, we don't want it to cache the pipeline, since caching the pipeline will likely result in producing the exact same results.

In [201]:
response = api_client.create_run_from_job_spec(
    job_spec_path="artifacts/reddit_scraper_pipeline_job.json",
    enable_caching=True  # Change to False when needing to generate new values per job run
)